# Imports

In [2]:
import re
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup


# Classic


In [3]:
#Category: Classic
url = 'https://books.toscrape.com/catalogue/category/books/classics_6/index.html'
# soup
headers = {'User-Agente': 'Mozilla/5.0 (Macintosh; Intel Max OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
url = requests.get(url, headers = headers)
soup = BeautifulSoup(url.text, 'html.parser')
books = soup.find('ol', class_ = 'row')

# Name
books_list = books.find_all('a')
books_name = [p.get('title') for p in books_list]
books_name = list(filter(None,books_name))
books_name = pd.DataFrame(books_name, columns = ['name'])

# Price
books_price = books.find_all('p', class_ = 'price_color')
books_price = [p.get_text() for p in books_price]
books_price = pd.DataFrame(books_price,columns=['price'])

# Rating
books_list = books.find_all('p',class_ = 'star-rating')
books_rating = [p.get('class') for p in books_list]
books_rating = pd.DataFrame(books_rating, columns= ['','rating'])
books_rating = books_rating.drop(columns = '')
books_rating['rating'] = books_rating['rating'].apply(lambda x: '1' if x == 'One' else '2' if x == 'Two' else '3' if x == 'Three' else '4' if x == 'Four' else '5')

# URL

books_url = books.find_all('a')
books_url = [p.get('href') for p in books_url]
books_url = list(set(books_url))
books_url = [p.replace('../../../','/') for p in books_url]
books_url = pd.DataFrame(books_url, columns = ['url'])

# Stock
stock_number_list = []
stock_name_list = []
for i in range(len(books_url)):
    url = 'https://books.toscrape.com/catalogue/' + books_url.loc[i, 'url']
    headers = {'User-Agente': 'Mozilla/5.0 (Macintosh; Intel Max OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    stock_number = requests.get(url, headers = headers )
    stock_number.text
    soup = BeautifulSoup(stock_number.text, 'html.parser')
    stock_number = soup.find_all('p', class_ = 'instock availability')
    stock_number = [p.get_text() for p in stock_number][0]
    stock_number_list.append(stock_number)
    stock_number_list = [p.replace('\n\n    \n        ','') for p in stock_number_list]
    stock_number_list = [p.replace('\n    \n','') for p in stock_number_list]
  
    stock_name = soup.find_all('h1')
    stock_name = [p.get_text() for p in stock_name]
    stock_name_list.append(stock_name)
        
    stock_name = pd.DataFrame(stock_name_list)
    stock_name.columns = ['name']    
    stock_number = pd.DataFrame(stock_number_list)
    stock_number.columns = ['stock']
    stock_number['category'] = 'classic'
    
    
classic_partial1 = pd.concat([books_name, books_price, books_rating],axis = 1)   
classic_partial2 = pd.concat([stock_name,stock_number],axis = 1)  
classic          = pd.merge(classic_partial1,classic_partial2, how = 'left',  on = 'name')

# Science Fiction


In [4]:
#Category: Science Fiction
url = 'https://books.toscrape.com/catalogue/category/books/science-fiction_16/index.html'
# soup
headers = {'User-Agente': 'Mozilla/5.0 (Macintosh; Intel Max OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
url = requests.get(url, headers = headers)
soup = BeautifulSoup(url.text, 'html.parser')
books = soup.find('ol', class_ = 'row')

# Name
books_list = books.find_all('a')
books_name = [p.get('title') for p in books_list]
books_name = list(filter(None,books_name))
books_name = pd.DataFrame(books_name, columns = ['name'])

# Price
books_price = books.find_all('p', class_ = 'price_color')
books_price = [p.get_text() for p in books_price]
books_price = pd.DataFrame(books_price,columns=['price'])

# Rating
books_list = books.find_all('p',class_ = 'star-rating')
books_rating = [p.get('class') for p in books_list]
books_rating = pd.DataFrame(books_rating, columns= ['','rating'])
books_rating = books_rating.drop(columns = '')
books_rating['rating'] = books_rating['rating'].apply(lambda x: '1' if x == 'One' else '2' if x == 'Two' else '3' if x == 'Three' else '4' if x == 'Four' else '5')

# URL

books_url = books.find_all('a')
books_url = [p.get('href') for p in books_url]
books_url = list(set(books_url))
books_url = [p.replace('../../../','/') for p in books_url]
books_url = pd.DataFrame(books_url, columns = ['url'])

# Stock
stock_number_list = []
stock_name_list = []
for i in range(len(books_url)):
    url = 'https://books.toscrape.com/catalogue/' + books_url.loc[i, 'url']
    headers = {'User-Agente': 'Mozilla/5.0 (Macintosh; Intel Max OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    stock_number = requests.get(url, headers = headers )
    stock_number.text
    soup = BeautifulSoup(stock_number.text, 'html.parser')
    stock_number = soup.find_all('p', class_ = 'instock availability')
    stock_number = [p.get_text() for p in stock_number][0]
    stock_number_list.append(stock_number)
    stock_number_list = [p.replace('\n\n    \n        ','') for p in stock_number_list]
    stock_number_list = [p.replace('\n    \n','') for p in stock_number_list]
  
    stock_name = soup.find_all('h1')
    stock_name = [p.get_text() for p in stock_name]
    stock_name_list.append(stock_name)
        
    stock_name = pd.DataFrame(stock_name_list)
    stock_name.columns = ['name']    
    stock_number = pd.DataFrame(stock_number_list)
    stock_number.columns = ['stock']
    stock_number['category'] = 'science_fiction'
    
sciencefiction_partial1 = pd.concat([books_name, books_price, books_rating],axis = 1)   
sciencefiction_partial2 = pd.concat([stock_name,stock_number],axis = 1)  
sciencefiction          = pd.merge(sciencefiction_partial1,sciencefiction_partial2, how = 'left',  on = 'name')

# Humor

In [5]:
#Category: Classic
url = 'https://books.toscrape.com/catalogue/category/books/humor_30/index.html'
# soup
headers = {'User-Agente': 'Mozilla/5.0 (Macintosh; Intel Max OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
url = requests.get(url, headers = headers)
soup = BeautifulSoup(url.text, 'html.parser')
books = soup.find('ol', class_ = 'row')

# Name
books_list = books.find_all('a')
books_name = [p.get('title') for p in books_list]
books_name = list(filter(None,books_name))
books_name = pd.DataFrame(books_name, columns = ['name'])

# Price
books_price = books.find_all('p', class_ = 'price_color')
books_price = [p.get_text() for p in books_price]
books_price = pd.DataFrame(books_price,columns=['price'])

# Rating
books_list = books.find_all('p',class_ = 'star-rating')
books_rating = [p.get('class') for p in books_list]
books_rating = pd.DataFrame(books_rating, columns= ['','rating'])
books_rating = books_rating.drop(columns = '')
books_rating['rating'] = books_rating['rating'].apply(lambda x: '1' if x == 'One' else '2' if x == 'Two' else '3' if x == 'Three' else '4' if x == 'Four' else '5')

# URL

books_url = books.find_all('a')
books_url = [p.get('href') for p in books_url]
books_url = list(set(books_url))
books_url = [p.replace('../../../','/') for p in books_url]
books_url = pd.DataFrame(books_url, columns = ['url'])

# Stock
stock_number_list = []
stock_name_list = []
for i in range(len(books_url)):
    url = 'https://books.toscrape.com/catalogue/' + books_url.loc[i, 'url']
    headers = {'User-Agente': 'Mozilla/5.0 (Macintosh; Intel Max OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    stock_number = requests.get(url, headers = headers )
    stock_number.text
    soup = BeautifulSoup(stock_number.text, 'html.parser')
    stock_number = soup.find_all('p', class_ = 'instock availability')
    stock_number = [p.get_text() for p in stock_number][0]
    stock_number_list.append(stock_number)
    stock_number_list = [p.replace('\n\n    \n        ','') for p in stock_number_list]
    stock_number_list = [p.replace('\n    \n','') for p in stock_number_list]
  
    stock_name = soup.find_all('h1')
    stock_name = [p.get_text() for p in stock_name]
    stock_name_list.append(stock_name)
        
    stock_name = pd.DataFrame(stock_name_list)
    stock_name.columns = ['name']    
    stock_number = pd.DataFrame(stock_number_list)
    stock_number.columns = ['stock']
    stock_number['category'] = 'humor'
    
    
humor_partial1 = pd.concat([books_name, books_price, books_rating],axis = 1)   
humor_partial2 = pd.concat([stock_name,stock_number],axis = 1)  
humor          = pd.merge(humor_partial1,humor_partial2, how = 'left',  on = 'name')

In [6]:
humor

,name,price,rating,stock,category
0,The Long Haul (Diary of a Wimpy Kid #9),Â£44.07,1,In stock (14 available),humor
1,Old School (Diary of a Wimpy Kid #10),Â£11.83,5,In stock (14 available),humor
2,I Know What I'm Doing -- and Other Lies I Tell...,Â£25.98,4,In stock (14 available),humor
3,"Hyperbole and a Half: Unfortunate Situations, ...",Â£14.75,5,In stock (14 available),humor
4,Dress Your Family in Corduroy and Denim,Â£43.68,3,In stock (9 available),humor
5,Toddlers Are A**holes: It's Not Your Fault,Â£25.55,1,In stock (7 available),humor
6,When You Are Engulfed in Flames,Â£30.89,5,In stock (3 available),humor
7,Naked,Â£31.69,3,In stock (3 available),humor
8,"Lamb: The Gospel According to Biff, Christ's C...",Â£55.50,5,In stock (3 available),humor
9,Holidays on Ice,Â£51.07,2,In stock (3 available),humor


# Business

In [7]:
#Category: Classic
url = 'https://books.toscrape.com/catalogue/category/books/business_35/index.html'
# soup
headers = {'User-Agente': 'Mozilla/5.0 (Macintosh; Intel Max OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
url = requests.get(url, headers = headers)
soup = BeautifulSoup(url.text, 'html.parser')
books = soup.find('ol', class_ = 'row')

# Name
books_list = books.find_all('a')
books_name = [p.get('title') for p in books_list]
books_name = list(filter(None,books_name))
books_name = pd.DataFrame(books_name, columns = ['name'])

# Price
books_price = books.find_all('p', class_ = 'price_color')
books_price = [p.get_text() for p in books_price]
books_price = pd.DataFrame(books_price,columns=['price'])

# Rating
books_list = books.find_all('p',class_ = 'star-rating')
books_rating = [p.get('class') for p in books_list]
books_rating = pd.DataFrame(books_rating, columns= ['','rating'])
books_rating = books_rating.drop(columns = '')
books_rating['rating'] = books_rating['rating'].apply(lambda x: '1' if x == 'One' else '2' if x == 'Two' else '3' if x == 'Three' else '4' if x == 'Four' else '5')

# URL

books_url = books.find_all('a')
books_url = [p.get('href') for p in books_url]
books_url = list(set(books_url))
books_url = [p.replace('../../../','/') for p in books_url]
books_url = pd.DataFrame(books_url, columns = ['url'])

# Stock
stock_number_list = []
stock_name_list = []
for i in range(len(books_url)):
    url = 'https://books.toscrape.com/catalogue/' + books_url.loc[i, 'url']
    headers = {'User-Agente': 'Mozilla/5.0 (Macintosh; Intel Max OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    stock_number = requests.get(url, headers = headers )
    stock_number.text
    soup = BeautifulSoup(stock_number.text, 'html.parser')
    stock_number = soup.find_all('p', class_ = 'instock availability')
    stock_number = [p.get_text() for p in stock_number][0]
    stock_number_list.append(stock_number)
    stock_number_list = [p.replace('\n\n    \n        ','') for p in stock_number_list]
    stock_number_list = [p.replace('\n    \n','') for p in stock_number_list]
  
    stock_name = soup.find_all('h1')
    stock_name = [p.get_text() for p in stock_name]
    stock_name_list.append(stock_name)
        
    stock_name = pd.DataFrame(stock_name_list)
    stock_name.columns = ['name']    
    stock_number = pd.DataFrame(stock_number_list)
    stock_number.columns = ['stock']
    stock_number['category'] = 'business'
    
    
business_partial1 = pd.concat([books_name, books_price, books_rating],axis = 1)   
business_partial2 = pd.concat([stock_name,stock_number],axis = 1)  
business          = pd.merge(business_partial1,business_partial2, how = 'left',  on = 'name')

# Dataset Final

In [8]:
books = pd.concat([classic,sciencefiction,humor,business]).reset_index(drop=True)

In [9]:
books.to_csv('books_catalogue.csv')